<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/achatbot_step_tts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install

In [ ]:
!git clone https://github.com/ai-bot-pro/achatbot.git -b feat/tts


In [2]:
%cd /content/achatbot


/content/achatbot


In [ ]:
!git submodule update --init --recursive


In [ ]:
!git checkout . && git pull origin feat/tts


In [ ]:
!bash -x scripts/pypi_achatbot.sh dev


In [ ]:
!pip install -q dist/achatbot-0.0.8.12-py3-none-any.whl[tts_step,grpc]


In [ ]:
!pip install onnxruntime-gpu==1.20.1

In [ ]:
!apt install -y sox

# download ckpt


In [8]:
from google.colab import userdata
HF_TOKEN=userdata.get('HF_TOKEN')

In [ ]:
!huggingface-cli login --token $HF_TOKEN --add-to-git-credential


In [10]:
!huggingface-cli download stepfun-ai/Step-Audio-Tokenizer --quie --local-dir /content/models/stepfun-ai/Step-Audio-Tokenizer


/content/models/stepfun-ai/Step-Audio-Tokenizer


In [11]:
!huggingface-cli download stepfun-ai/Step-Audio-TTS-3B --quie --local-dir /content/models/stepfun-ai/Step-Audio-TTS-3B


/content/models/stepfun-ai/Step-Audio-TTS-3B


# test tts


In [12]:
%cd /content/achatbot


/content/achatbot


In [ ]:
!LOG_LEVEL=info TTS_LM_MODEL_PATH=/content/models/stepfun-ai/Step-Audio-TTS-3B \
  TTS_TOKENIZER_MODEL_PATH=/content/models/stepfun-ai/Step-Audio-Tokenizer \
  python -m unittest test.modules.speech.tts.test_step.TestStepTTS.test_get_voices

In [ ]:
!LOG_LEVEL=info TTS_LM_MODEL_PATH=/content/models/stepfun-ai/Step-Audio-TTS-3B \
  TTS_TOKENIZER_MODEL_PATH=/content/models/stepfun-ai/Step-Audio-Tokenizer \
  REF_AUDIO_PATH=./test/audio_files/asr_example_zh.wav \
  REF_TEXT="欢迎大家来体验达摩院推出的语音识别模型" \
  python -m unittest test.modules.speech.tts.test_step.TestStepTTS.test_set_voice

In [ ]:
!LOG_LEVEL=debug TTS_WARMUP_STEPS=2 TTS_LM_MODEL_PATH=/content/models/stepfun-ai/Step-Audio-TTS-3B \
  TTS_TOKENIZER_MODEL_PATH=/content/models/stepfun-ai/Step-Audio-Tokenizer \
  python -m unittest test.modules.speech.tts.test_step.TestStepTTS.test_synthesize

In [ ]:
from IPython.display import Audio
Audio("/content/achatbot/records/test_tts_step.wav",autoplay=True)

In [ ]:
# ref audio
!LOG_LEVEL=debug TTS_WARMUP_STEPS=2 TTS_LM_MODEL_PATH=/content/models/stepfun-ai/Step-Audio-TTS-3B \
  TTS_TOKENIZER_MODEL_PATH=/content/models/stepfun-ai/Step-Audio-Tokenizer \
  TTS_STREAM_FACTOR=5 \
  REF_AUDIO_PATH=./test/audio_files/asr_example_zh.wav \
  REF_TEXT="欢迎大家来体验达摩院推出的语音识别模型" \
  TTS_TEXT="万物之始,大道至简,衍化至繁。君不见黄河之水天上来，奔流到海不复回。君不见高堂明镜悲白发，朝如青丝暮成雪。人生得意须尽欢，莫使金樽空对月。天生我材必有用，千金散尽还复来。" \
  python -m unittest test.modules.speech.tts.test_step.TestStepTTS.test_synthesize

In [ ]:
from IPython.display import Audio
Audio("/content/achatbot/records/test_tts_step.wav",autoplay=True)

In [ ]:
# ---- TTS_MODE: voice_clone ----
# src audio + default ref audio
!LOG_LEVEL=debug TTS_WARMUP_STEPS=2 TTS_LM_MODEL_PATH=/content/models/stepfun-ai/Step-Audio-TTS-3B \
  TTS_TOKENIZER_MODEL_PATH=/content/models/stepfun-ai/Step-Audio-Tokenizer \
  TTS_STREAM_FACTOR=2 \
  TTS_MODE=voice_clone \
  SRC_AUDIO_PATH=./test/audio_files/asr_example_zh.wav \
  python -m unittest test.modules.speech.tts.test_step.TestStepTTS.test_synthesize

In [ ]:
from IPython.display import Audio
Audio("/content/achatbot/records/test_tts_step.wav",autoplay=True)

# tts speaker with grpc


In [77]:
%cd /content/achatbot


/content/achatbot


In [ ]:
#!python -m src.cmd.grpc.speaker.server.serve


In [ ]:
!TTS_TAG=tts_step IS_SAVE=1 IS_RELOAD=1 \
  TTS_WARMUP_STEPS=2 TTS_LM_MODEL_PATH=/content/models/stepfun-ai/Step-Audio-TTS-3B \
  TTS_TOKENIZER_MODEL_PATH=/content/models/stepfun-ai/Step-Audio-Tokenizer \
  python -m src.cmd.grpc.speaker.client

In [ ]:
from IPython.display import Audio
Audio("/content/achatbot/records/grpc_tts_step.wav",autoplay=True)

In [87]:
!TTS_TAG=tts_step IS_SAVE=1 IS_RELOAD=1 \
  TTS_WARMUP_STEPS=2 TTS_LM_MODEL_PATH=/content/models/stepfun-ai/Step-Audio-TTS-3B \
  TTS_TOKENIZER_MODEL_PATH=/content/models/stepfun-ai/Step-Audio-Tokenizer \
  TTS_STREAM_FACTOR=2 \
  TTS_MODE=voice_clone \
  SRC_AUDIO_PATH=./test/audio_files/asr_example_zh.wav \
  python -m src.cmd.grpc.speaker.client

2025-02-21 11:25:25,761 - chat-bot-tts-client - DEBUG - /content/achatbot/src/cmd/grpc/speaker/client.py:59 - load_model - tts_tag: "tts_step"
is_reload: true
json_kwargs: "{\"lm_args\": {\"lm_model_name_or_path\": \"/content/models/stepfun-ai/Step-Audio-TTS-3B\", \"lm_device_map\": null, \"lm_device\": null, \"lm_torch_dtype\": \"auto\", \"lm_attn_impl\": null, \"user_role\": \"user\", \"warnup_prompt\": \"Repeat the word \'weedge niu bi\'.\", \"warmup_steps\": 2, \"init_chat_role\": \"system\", \"init_chat_prompt\": \"You are a helpful and friendly AI assistant. You are polite, respectful, and aim to provide concise responses of less than 20 words.\", \"lm_gen_seed\": 42, \"lm_gen_max_new_tokens\": 1024, \"lm_gen_min_new_tokens\": 0, \"lm_gen_do_sample\": true, \"lm_gen_temperature\": 0.8, \"lm_gen_top_k\": 10, \"lm_gen_top_p\": 1.0, \"lm_gen_repetition_penalty\": 1.1, \"lm_tokenizer_decode_batch_size\": 60, \"chat_history_size\": null, \"lm_stream\": true, \"model_type\": \"chat_com

In [ ]:
from IPython.display import Audio
Audio("/content/achatbot/records/grpc_tts_step.wav",autoplay=True)